In [1]:
import os
import numpy as np
import pretty_midi
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, MultiHeadAttention, Input, Embedding
import tensorflow as tf
import math

In [2]:
# Configuration de base
sequence_size = 100  # Nombre de notes et durées par séquence
# Increase by 1 to account for the shift when creating x/y pairs
embed_dim = 128
num_heads = 2
feed_forward_dim = 256
batch_size = 16
# Augmentez la taille du vocabulaire pour couvrir les notes potentiellement plus larges
vocab_size = 156  # Choisi pour couvrir la plage de notes inattendue dans vos fichiers

## 3.b. Chargement du dataset Jazz Midi

In [3]:
# 3. Chargement du dataset Jazz Midi
midi_dir = "./Jazz Midi"
midi_files = [os.path.join(midi_dir, f) for f in os.listdir(midi_dir) if f.endswith(".mid")]

def load_midi(file_path):
    """Charge un fichier MIDI et retourne des listes de notes et de durées."""
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        notes, durations = [], []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    notes.append(note.pitch)
                    durations.append(note.end - note.start)
        return notes, durations
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return [], []  # Return empty lists if file can't be loaded

# Modified file loading loop
notes_list, durations_list = [], []
for file in midi_files:
    notes, durations = load_midi(file)
    if notes and durations:  # Only append if we got valid data
        notes_list.append(notes)
        durations_list.append(durations)

/opt/anaconda3/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error loading ./Jazz Midi/Lakes.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/StTropez.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/LovinTouchinSqueezin.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/Moment.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/AnyWayYouWantIt.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/Destiny.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/JamaicanNights.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/AffairInSanMiguel.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/TheCloserIGetToYou.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/CurvesAhead.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/CantilopeIsland.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/ThePrincess.mid: data byte must be in range 0..127
Error loading ./Jazz Midi/NativeSonsOfADistantLand.mid: data 

## 4.b. Créer deux listes pour chaque morceau, la première avec la succession des notes et la seconde avec la durée de chacune d'elles.

In [4]:
# 4. Créer deux listes pour chaque morceau, la première avec la succession des notes et la seconde avec la durée de chacune d'elles.

# Trouver la valeur maximale dans les séquences de notes pour ajuster vocab_size
max_note_value = max([max(notes) for notes in notes_list if notes])
vocab_size = max_note_value + 1  # Ajuster vocab_size en fonction de la note maximale

print(f"Vocab size ajusté : {vocab_size}")

# Fonction pour limiter les valeurs de notes à vocab_size - 1 si besoin
def limit_note_values(data, max_value):
    return [[min(note, max_value) for note in seq] for seq in data]


notes_list = limit_note_values(notes_list, vocab_size - 1)

# Vectorisation et segmentation en séquences de taille fixe
def create_sequences(data, sequence_length=sequence_size + 1):
    sequences = []
    for item in data:
        # Limitez les valeurs pour qu'elles soient dans [0, vocab_size - 1]
        item = [min(note, vocab_size - 1) for note in item]
        for i in range(0, len(item) - sequence_length, sequence_length):
            sequences.append(item[i:i + sequence_length])
    return sequences

# Recréer les séquences de notes et durées en tenant compte du vocab_size mis à jour
notes_sequences = create_sequences(notes_list)
durations_sequences = create_sequences(durations_list)

Vocab size ajusté : 128


## 5.b. Vectoriser séparément chacune des deux listes précédentes et créer des données d'un nombre fixé d'éléments (par exemple 100 ou 200, à voir selon vos ressources).

In [5]:
# 5. Vectoriser séparément chacune des deux listes précédentes et créer des données d'un nombre fixé d'éléments (par exemple 100 ou 200, à voir selon vos ressources).
# Conversion en tenseurs
notes_sequences = tf.convert_to_tensor(notes_sequences, dtype=tf.int32)
durations_sequences = tf.convert_to_tensor(durations_sequences, dtype=tf.float32)

## 6.b. Créer le jeu d'entraînement (en s'inspirant de ce qui a été fait avec des textes pendant le cours).

In [6]:
# 6. Créer le jeu d'entraînement (en s'inspirant de ce qui a été fait avec des textes pendant le cours).
# Création du jeu d'entraînement (décalage de 1 pour la prédiction de la prochaine note)
x_notes, y_notes = notes_sequences[:, :-1], notes_sequences[:, 1:]
x_durations, y_durations = durations_sequences[:, :-1], durations_sequences[:, 1:]

## 7.b. Implémenter une fonction de masquage

In [7]:
# 7. Implémenter une fonction de masquage
def causal_attention_mask(batch_size, n_dest, n_src):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    mask = i >= j - n_src + n_dest
    mask = tf.cast(mask, dtype=tf.bool)
    return tf.tile(mask[None, :, :], [batch_size, 1, 1])

## 8.b. Embedding et Positional Encoding

In [8]:
# 8. Embedding et Positional Encoding
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, sequence_size, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.sequence_size = sequence_size
        self.embed_dim = embed_dim

    def call(self, x):
        x = self.token_emb(x)
        pe = np.zeros((self.sequence_size, self.embed_dim))
        for pos in range(self.sequence_size):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (2 * i / self.embed_dim)))
                pe[pos, i+1] = math.cos(pos / (10000 ** (2 * i / self.embed_dim)))
        pe = tf.convert_to_tensor(pe, dtype=tf.float32)
        return x + pe

## 9.b. Bloc Transformer avec concaténation des notes et durées

In [9]:
# 9. Bloc Transformer avec concaténation des notes et durées
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):  # Made training parameter optional with default False
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len)
        attn_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class ConcatenationLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def call(self, inputs):
        return tf.concat(inputs, axis=-1)

class StackedTransformerModel:
    def __init__(self, num_transformer_blocks=4):
        self.num_transformer_blocks = num_transformer_blocks

    def create_model(self):
        notes_input = Input(shape=(sequence_size,), dtype="int32")  # Séquence de notes
        durations_input = Input(shape=(sequence_size,), dtype="float32")  # Séquence de durées

        # Embedding avec encodage positionnel
        embedding_layer = TokenAndPositionEmbedding(sequence_size, vocab_size=vocab_size, embed_dim=embed_dim)
        notes_embedded = embedding_layer(notes_input)
        durations_embedded = embedding_layer(durations_input)

        # Concaténation des embeddings de notes et durées
        concat_layer = ConcatenationLayer()
        concat_inputs = concat_layer([notes_embedded, durations_embedded])
        
        # Application de plusieurs blocs Transformer
        transformer_output = concat_inputs
        for _ in range(self.num_transformer_blocks):
            transformer_block = TransformerBlock(embed_dim * 2, num_heads, feed_forward_dim)
            transformer_output = transformer_block(transformer_output)

        # Sorties pour les prédictions des notes et des durées
        notes_output = Dense(vocab_size, activation="softmax")(transformer_output)
        durations_output = Dense(1)(transformer_output)

        model = Model(inputs=[notes_input, durations_input], outputs=[notes_output, durations_output])
        model.compile(optimizer="adam", loss=["sparse_categorical_crossentropy", "mse"])
        return model

# Création du modèle avec plusieurs blocs Transformer
num_transformer_blocks = 4  # Ajuster ce nombre pour empiler les blocs
stacked_model = StackedTransformerModel(num_transformer_blocks)
model = stacked_model.create_model()

## 10.b. Entraînement du réseau

In [10]:
### 10. Entraînement du réseau
model.fit([x_notes, x_durations], [y_notes, y_durations], batch_size=batch_size, epochs=10)

Epoch 1/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 420s 251ms/step - dense_8_loss: 4.0046 - dense_9_loss: 1.4633 - loss: 5.4680
Epoch 2/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 404s 244ms/step - dense_8_loss: 3.6365 - dense_9_loss: 0.6011 - loss: 4.2376
Epoch 3/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 395s 238ms/step - dense_8_loss: 3.3827 - dense_9_loss: 0.4619 - loss: 3.8446
Epoch 4/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 403s 243ms/step - dense_8_loss: 3.4013 - dense_9_loss: 0.4988 - loss: 3.9001
Epoch 5/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 401s 242ms/step - dense_8_loss: 3.8748 - dense_9_loss: 0.5370 - loss: 4.4118
Epoch 6/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 404s 244ms/step - dense_8_loss: 3.5305 - dense_9_loss: 0.5954 - loss: 4.1260
Epoch 7/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 384s 232ms/step - dense_8_loss: 3.4733 - dense_9_loss: 0.5993 - loss: 4.0726
Epoch 8/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 381s 230ms/step - dense_8_loss: 3.5410 - dense_9_loss: 0.5715 - loss: 4.1125
Epoch 9/10
1657/1657 ━━━━━━━━━━━━━━━━━━━━ 381s 2

## 11.b. Utiliser ce réseau pour générer de nouveaux morceaux (on fera un tirage aléatoire avec les probabilités retournées par la dernière couche). Les convertir en un morceau audible.
## 12.b. Rajouter un terme de température. Étudier son effet.

In [11]:
### 11. Génération de nouveaux morceaux et 12. Effet de la température
import os
import pretty_midi
import numpy as np
import tensorflow as tf

# Créer le dossier "generated_midis" s'il n'existe pas
output_dir = "generated_midis"
os.makedirs(output_dir, exist_ok=True)

# Fonction mise à jour pour générer de la musique et sauvegarder en MIDI
def generate_music(model, start_notes, start_durations, num_notes=100, temperature=1.0, output_path="generated_music.mid"):
    generated_notes, generated_durations = [], []
    for _ in range(num_notes):
        pred_notes, pred_durations = model.predict([start_notes, start_durations])
        
        # Échantillonner la note suivante
        next_note = tf.random.categorical(pred_notes[0] / temperature, num_samples=1).numpy()[0]
        next_duration = max(0, pred_durations[0, 0])  # Durée positive uniquement
        
        # Ajouter les prédictions aux séquences générées
        generated_notes.append(next_note)
        generated_durations.append(next_duration)
        
        # Convertir les tenseurs en int32 et float32 avant concaténation
        start_notes = tf.concat([tf.cast(start_notes[:, 1:], tf.int32), tf.expand_dims(tf.cast(next_note, tf.int32), 0)], axis=1)
        start_durations = tf.concat([start_durations[:, 1:], tf.expand_dims(tf.cast(next_duration, tf.float32), 0)], axis=1)
    
    # Création du fichier MIDI
    midi_data = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Piano par défaut
    current_time = 0  # Temps de démarrage pour la première note
    
    for pitch, duration in zip(generated_notes, generated_durations):
        # Ajouter chaque note à l'instrument
        note = pretty_midi.Note(
            velocity=100,  # Intensité de la note
            pitch=int(pitch),  # Hauteur de la note
            start=current_time,
            end=current_time + float(duration)
        )
        instrument.notes.append(note)
        current_time += float(duration)  # Mettre à jour le temps pour la prochaine note
    
    midi_data.instruments.append(instrument)
    
    # Sauvegarder le fichier MIDI dans le dossier "generated_midis"
    output_file = os.path.join(output_dir, output_path)
    midi_data.write(output_file)
    print(f"Morceau généré et sauvegardé sous {output_file}")

# Exemple d'utilisation avec des séquences de départ
start_notes = tf.constant([[60] * sequence_size], dtype=tf.int32)  # Commencer avec une note de base, par ex. '60' (do central)
start_durations = tf.constant([[0.5] * sequence_size], dtype=tf.float32)  # Durée de 0.5 pour chaque note initiale

# Générer et sauvegarder le morceau
generate_music(model, start_notes, start_durations, num_notes=100, temperature=1.0, output_path="02-generated_music.mid")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━

/var/folders/1k/1sj3zxpx1x78x8blphwrrcrc0000gn/T/ipykernel_64385/884711939.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pitch=int(pitch),  # Hauteur de la note
/var/folders/1k/1sj3zxpx1x78x8blphwrrcrc0000gn/T/ipykernel_64385/884711939.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  end=current_time + float(duration)
/var/folders/1k/1sj3zxpx1x78x8blphwrrcrc0000gn/T/ipykernel_64385/884711939.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_ti

In [13]:
import IPython.display as ipd
import soundfile as sf
import pretty_midi

output_file = "./generated_midis/02-generated_music.mid"
midi_data = pretty_midi.PrettyMIDI(output_file)

audio_file = "./generated_midis/02-generated_music.wav"
waveform = midi_data.synthesize()
sf.write(audio_file, waveform, samplerate=44100)

ipd.Audio(audio_file)

## 14.b. Sauvegarder le modèle

In [12]:
### 14. Sauvegarde du modèle
model.save("02-jazz_transformer_model.keras")